In [1]:
from icepyx import icesat2data as ipd
import os
import shutil
from pprint import pprint
%matplotlib inline

In [29]:
short_name = 'ATL06'
spatial_extent = [ -50.71, -45.32, 68.08, 65.73]
date_range = ['2019-06-01','2019-06-05']

In [30]:
region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range)

print(region_a.dataset)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.dataset_version)
print(region_a.spatial_extent)

ATL06
['2019-06-01', '2019-06-05']
00:00:00
23:59:59
003
['bounding box', [-50.71, -45.32, 68.08, 65.73]]


In [31]:
region_a.avail_granules(ids=True)

['ATL06_20190601005859_09740303_003_01.h5',
 'ATL06_20190601023316_09750303_003_01.h5',
 'ATL06_20190601035904_09760302_003_01.h5',
 'ATL06_20190601054151_09770303_003_01.h5',
 'ATL06_20190601071608_09780303_003_01.h5',
 'ATL06_20190601120935_09810305_003_01.h5',
 'ATL06_20190601121500_09810306_003_01.h5',
 'ATL06_20190601134352_09820305_003_01.h5',
 'ATL06_20190601134917_09820306_003_01.h5',
 'ATL06_20190601140450_09820308_003_01.h5',
 'ATL06_20190601151810_09830305_003_01.h5',
 'ATL06_20190601182644_09850305_003_01.h5',
 'ATL06_20190601200102_09860305_003_01.h5',
 'ATL06_20190602000443_09880313_003_01.h5',
 'ATL06_20190602002449_09890302_003_01.h5',
 'ATL06_20190602003320_09890303_003_01.h5',
 'ATL06_20190602020737_09900303_003_01.h5',
 'ATL06_20190602033324_09910302_003_01.h5',
 'ATL06_20190602034154_09910303_003_01.h5',
 'ATL06_20190602065029_09930303_003_01.h5',
 'ATL06_20190602114355_09960305_003_01.h5',
 'ATL06_20190602114920_09960306_003_01.h5',
 'ATL06_20190602131813_09970305_

In [35]:
earthdata_uid = 'tsnow03'
email = 'tasha.snow@colorado.edu'
region_a.earthdata_login(earthdata_uid, email)

Earthdata Login password:  ············


In [36]:
region_a.order_vars.avail(options=True)

var_list inputs: atlas_sdp_gps_epoch, control, data_end_utc, data_start_utc, end_cycle, end_delta_time, end_geoseg, end_gpssow, end_gpsweek, end_orbit, end_region, end_rgt, granule_end_utc, granule_start_utc, qa_at_interval, release, start_cycle, start_delta_time, start_geoseg, start_gpssow, start_gpsweek, start_orbit, start_region, start_rgt, version, dt_hist, fit_maxiter, fpb_maxiter, maxiter, max_res_ids, min_dist, min_gain_th, min_n_pe, min_n_sel, min_signal_conf, n_hist, nhist_bins, n_sigmas, proc_interval, qs_lim_bsc, qs_lim_hrs, qs_lim_hsigma, qs_lim_msw, qs_lim_snr, qs_lim_sss, rbin_width, sigma_beam, sigma_tx, t_dead, atl06_quality_summary, delta_time, h_li, h_li_sigma, latitude, longitude, segment_id, sigma_geo_h, fpb_mean_corr, fpb_mean_corr_sigma, fpb_med_corr, fpb_med_corr_sigma, fpb_n_corr, med_r_fit, tx_mean_corr, tx_med_corr, dem_flag, dem_h, geoid_h, dh_fit_dx, dh_fit_dx_sigma, dh_fit_dy, h_expected_rms, h_mean, h_rms_misfit, h_robust_sprd, n_fit_photons, n_seg_pulses,

In [37]:
region_a.order_vars.append(beam_list=['gt1l', 'gt2l', 'gt3l'],var_list=['longitude','latitude','atl06_quality_summary'])
pprint(region_a.order_vars.wanted)

ValueError: You must construct a wanted variable list in order to remove values from it.

In [ ]:
region_a.subsetparams(Coverage=region_a.order_vars.wanted)

In [ ]:
# View a short list of order IDs:
region_a.granules.orderIDs